**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

<font color='red' size="4">Import any additional libraries you need in the same code block that you use it.</font>

In [ ]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## Question 1b - Review the relevant code samples provided on Canvas. Note: SimonX/Y indicates block size X and key size Y. The same applies to SpeckX/Y. Then, using Python crypto libraries in a secure way, implement the following:

In [ ]:
#! pip install SimonSpeck
#! pip install pycryptodome

from Crypto.Cipher import AES
from Crypto.Hash import SHA256, CMAC
from speck import SpeckCipher
from Crypto.Random import get_random_bytes
from Crypto.Util.number import bytes_to_long , long_to_bytes
import hmac
import secrets

In [41]:
from Cryptodome.Cipher import AES
from Cryptodome.Hash import SHA256, CMAC
from speck import SpeckCipher
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.number import bytes_to_long , long_to_bytes
import hmac
import secrets

### Q1b [15 points] Implement MtE using Speck128/128-CBC (with a random IV) as the cipher and HMAC-SHA-256 as the MAC. Your implementation should support multiple plaintext blocks, and include decryption & mac verification support.

In [42]:
# 5 points for tag generation/verification

def mac_hmacsha(message: bytes, key: bytes) -> bytes:
    """ 
    Securely generate the HMAC-SHA256 Tag with the hmac builtin function using the
    SHA256 hash library. Return the tag in bytes
    """
    tag = b""
    # YOUR CODE HERE
    tag=hmac.new(key,message,SHA256).digest() # tag 
    return tag

def mac_hmacsha_verify(tag: bytes, message: bytes, key: bytes) -> bool:
    """ 
    Securely verify the HMAC-SHA256 Tag with the hmac builtin function using the
    SHA256 hash library. Return True if MAC is valid, else return False. 
    """
    # YOUR CODE HERE
    calculated_tag=hmac.new(key,message,SHA256).digest()
    return hmac.compare_digest(calculated_tag,tag)# verify tag

In [43]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
tag = mac_hmacsha(ptxt, key)
assert tag == b'E\xd7\n\x16U\xca\x13\xbcz\x9a\x93z\x7f\x7f\xf3\xe3\x8f\x1b<.\xc7j\r\xfb\xbe\xf1\xd2)\xc7\xd6\x08\x8f'
verify = mac_hmacsha_verify(tag, ptxt, key)
assert verify == True

In [44]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
tag = b'E\xd7\n\x16U\xaa\x13\xbcz\x9a\x93z\x7f\x7f\xf3\xe3\x8f\x1b<.\xc7j\r\xfb\xbe\xf1\xd2)\xc7\xd6\x08\x8f'
verify = mac_hmacsha_verify(tag, ptxt, key)
assert verify == False

In [45]:
# 5 points for encryption/decryption

def encrypt_speck(ptxt: bytes, key: bytes) -> (bytes, bytes):
    """ 
    Encrypt the ptxt using Speck128/128-CBC. Use the given key.
    Return the ctxt and generated iv as bytes.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions
    """
    ctxt= b""
    iv = b""
    # YOUR CODE HERE
    iv=secrets.token_bytes(16)  # generate iv
    cipher=SpeckCipher(bytes_to_long(key), key_size=128, block_size=128, mode='CBC', init=bytes_to_long(iv)) # initialise speck cipher 
    block_size=16 # block size
    padded_ptxt=ptxt + b'\x00' * (block_size - len(ptxt) % block_size)  # add padding
    for i in range(0, len(padded_ptxt), block_size):
        block=padded_ptxt[i:i + block_size] # get block
        ctxt+=long_to_bytes(cipher.encrypt(bytes_to_long(block)))

    return (ctxt, iv)
def decrypt_speck(ctxt: bytes, iv: bytes, key: bytes) -> bytes:
    """ 
    Decrypt the ctxt using Speck128/128-CBC. Use the given key and iv. 
    Return the decrypted ptxt.
    Use long_to_bytes() and bytes_to_long() for bytes <-> int conversions
    """
    ptxt = b""
    # YOUR CODE HERE
    cipher=SpeckCipher(bytes_to_long(key), key_size=128, block_size=128, mode='CBC', init=bytes_to_long(iv)) # speck cipher
    block_size=16 # block size

    for i in range(0, len(ctxt), block_size):
        block=ctxt[i:i + block_size] # get block
        ptxt+=long_to_bytes(cipher.decrypt(bytes_to_long(block))) # decrypt ctxt

    return ptxt.rstrip(b'\x00') # remove zeros


In [46]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'
ctxt = b'/\xd6\xa5\x93$\xdf\x14\xa2\x9f\xf6\x90\xd6,\xa0\xc1SK\x84\xc9\x7f\xab\xdco\xd0C\xfd\x1a\xae\xcf\xdb\xddy3+\x0b\x8b\xa37\x8a\xf8\t\x1f\x16\\\x7fs\x1b\xca'
iv = b'\x92\xd1\xcfx\x0f\x8f\x99PBr\xe3.\xd5H\x164\xe8"\xd52g>\x885\x01\x83\x0b\xb0\x1c\xf9\x81\xd6\xe7\xa9\xbf\xfcB7D\x13\xbd\x97m\x10\xd9\x10l\x8c\xfa\x19\xf7JNq\x95\x06O\xb6S\x84\x96U:\xfc\xc4H\xea\xa8\xa6\x8d\xcf\x90=yH\x03\xf5\xc7\xff\x16/Q\x11\x8bT\x16\xf1R\xbe\xe0\xceG2_/\x91\xcb\xf1i\x89\xb5B\x012\xdf\xc2\xb8j\xe4\x0e;\x96ag\x82\tg\xd4#\xa6\x01hRD|\xa5\xbei'

ptxt_decrypt = decrypt_speck(ctxt, iv, key)
assert ptxt_decrypt == ptxt

In [47]:
ptxt = b"this  a random message that is sufficiently long"
key = b'\x0f?\xb9c1\xb6\x91\xe8\x8c\xc2\xbf\x10\xab\x12b\x98\x82\xf3h\xaa\x19! \xb99\x0fi1\xdc3\xb1\xcf'

ctxt, iv = encrypt_speck(ptxt, key)
ptxt_decrypt = decrypt_speck(ctxt, iv, key)
assert ptxt_decrypt == ptxt

# Ensure we generate a new iv
ctxt_new, iv_new = encrypt_speck(ptxt, key)
assert iv_new != iv
assert ctxt_new != ctxt

In [48]:
#[3 points] Implement you own MtE_encrypt() function and implement an example to demonstrate its use.
#You can have a hardcoded message, but the remaining elements must be securely generated.
#Print the ciphertext and tag result in hex.

In [49]:
# YOUR CODE HERE
def MtE_encrypt(message:bytes,enc_key:bytes,mac_key:bytes) -> (bytes,bytes,bytes,bytes):
    iv=secrets.token_bytes(16) # generate iv
    tag=mac_hmacsha(message,mac_key) # get tag
    ciphertext,iv=encrypt_speck(message+tag,enc_key) #encrypt message+tag
    return ciphertext,tag,iv

In [50]:
enc_key=secrets.token_bytes(16) # generate key
mac_key=secrets.token_bytes(16) # generate mac key
message = b"This is a secret message"
ciphertext, tag ,iv= MtE_encrypt(message,enc_key,mac_key)
print("Ciphertext:",ciphertext.hex())
print("Tag:",tag.hex())


Ciphertext: f45293f450b0596046bec33738ebb4b8b70b1dbe369826bd9b1264abdfc8ff57714e9f177c0a1486a669f08bacb13bd3523c4a435003b06d2b2fd2a180973528
Tag: d45aa163ed207fc4c6746a2498c841f1caa6de7c18bd1b8363f2c18f8bc0fa51


In [51]:
# hidden test cases

In [52]:
# [2 points] Implement you own MtE_decrypt() function. 
# Demostrate its use by decrypting and verifying the EaM_encrypt result above. 
# Print the plaintext as a byte string and tag result.

In [55]:
# YOUR CODE HERE
def MtE_decrypt(ciphertext: bytes, iv: bytes, enc_key: bytes, tag: bytes,mac_key:bytes) -> bytes:
    decrypted_data = decrypt_speck(ciphertext, iv, enc_key) # decrypt ciphertext
    message, tag = decrypted_data[:-32], decrypted_data[-32:] # split tag and ptxt
    if not mac_hmacsha_verify(tag, message, mac_key):
        raise ValueError("Invalid Tag!") # verify tag
    return message

In [58]:
enc_key=secrets.token_bytes(16) # generate key
mac_key=secrets.token_bytes(16) # generate mac key
message = b"This is a secret message"
ciphertext, tag,iv,enc_key,mac_key = MtE_encrypt(message,enc_key,mac_key) # mte encrypt
try:
        ptxt = MtE_decrypt(ciphertext, iv, enc_key,tag,mac_key) # mte decrypt
        print("Plaintext:",ptxt)
        print("Tag:",tag)
except ValueError as e:
        print(e)

Plaintext: b'This is a secret message'
Tag: b'\xb3\xb7\xf29V\xb8\xb4\x94tz\rI\xbe\xe5\x96\x01%\x92\xd9\x85\xd0\xfa\xda\xb3v\xaaZ\x19@\x87\xff\xad'


In [15]:
# hidden test cases